In [1]:
from PIL import Image
import numpy as np

import cv2
import pathlib
from tqdm import tqdm
import pickle
from colorthief import ColorThief
from matplotlib.pyplot import imshow

In [2]:
root = pathlib.Path('/data1/Raw')
out = pathlib.Path('/data2/ResizedImages/Depth10')

In [ ]:
all_images_paths = list(root.glob('**/*.jpg'))
all_image_data = []
wrong_sized_imgs = 0
count = 0
data_count = 0

In [4]:
for source in tqdm(all_images_paths):
    
    img = cv2.imread(str(source))
    org_array = np.asarray(img)
    
    if org_array.shape != (1080, 1920, 3):
        wrong_sized_imgs += 1
        
    count += 1
    
print(wrong_sized_imgs, count)

100%|██████████| 446054/446054 [3:55:15<00:00, 31.60it/s]   

313336 446054


In [5]:
depth = 10
size = 4*(2**(depth-1))
FHD_WIDTH = size
FHD_HEIGHT = size

def fill_background(img, dom_color):
    width = img.shape[1]
    height = img.shape[0]
    
    hmargin = int((FHD_WIDTH - width) / 2)
    vmargin = int((FHD_HEIGHT - height) / 2)

    output = cv2.copyMakeBorder(
                 img, 
                 vmargin, 
                 vmargin, 
                 hmargin, 
                 hmargin, 
                 cv2.BORDER_CONSTANT, 
                 value=dom_color
              )

    return output

def resize_image(img, w, h):
    resized = cv2.resize(img, (int(w),int(h)))
    return resized

def fix_image_size(img, source):
    width = img.shape[1]
    height = img.shape[0]
    
    # too large/small
    ratio1 = FHD_WIDTH/width
    ratio2 = FHD_HEIGHT/height

    ratio = min(ratio1, ratio2)

    img = resize_image(img, ratio*width, ratio*height)
    
    if img.shape == (1080, 1920, 3):
        return img
    
    dom_color = ColorThief(str(source)).get_color(quality=1)
    img = fill_background(img, dom_color)
    
    if img.shape == (1080, 1920, 3):
        return img
    
    return resize_image(img, FHD_WIDTH, FHD_HEIGHT)

In [6]:
save_img = None
for source in tqdm(all_images_paths):
    try:
        org_path = str(source).split('/')
        out_path = pathlib.Path(out, org_path[-3], org_path[-2], org_path[-1])

        if out_path.exists():
            img = cv2.imread(str(out_path))
            
            if img.shape == (1080, 1920, 3):
                continue
        
        img = cv2.imread(str(source))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        if img.shape != (1080, 1920, 3):
            img = fix_image_size(img, source)

        out_path.parent.mkdir(parents=True, exist_ok=True)
        cv2.imwrite(str(out_path), img)
    except Exception as e:
        print('Error', source, e)

  0%|          | 48/446054 [00:01<3:02:26, 40.75it/s]


KeyboardInterrupt: 

In [5]:
# Multiprocessing version

import multiprocessing
from multiprocessing import Process, Queue
import time
import queue

print("Number of cpu : ", multiprocessing.cpu_count())

Number of cpu :  8


In [6]:
img_queue = Queue()

for source in tqdm(all_images_paths):
    img_queue.put(source)

100%|██████████| 446054/446054 [00:00<00:00, 705683.45it/s]


In [8]:
def worker(img_queue):
    while True:
        try:
            source = img_queue.get_nowait()
            
            try:
                org_path = str(source).split('/')
                out_path = pathlib.Path(out, org_path[-3], org_path[-2], org_path[-1])

                if out_path.exists():
                    img = cv2.imread(str(out_path))

                    if img.shape == (1080, 1920, 3):
                        continue

                img = cv2.imread(str(source))
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

                if img.shape != (1080, 1920, 3):
                    img = fix_image_size(img, source)

                out_path.parent.mkdir(parents=True, exist_ok=True)
                cv2.imwrite(str(out_path), img)
            except Exception as e:
                print('Error', source, e)
        except queue.Empty:
            break
    return True

jobs=[]
for i in range(7):
    p = multiprocessing.Process(target=worker, args=(img_queue,))
    jobs.append(p)
    p.start()
    
for j in jobs:
    j.join()

Error /data1/Raw/WallpaperPlay/initial-d-wallpapers/84744.jpg OpenCV(4.3.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'



Process Process-4:
Traceback (most recent call last):
  File "/home/andrew/anaconda3/envs/gogh-get-money/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()


KeyboardInterrupt: 

  File "/home/andrew/anaconda3/envs/gogh-get-money/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-8-c95174a58eb7>", line 11, in worker
    img = cv2.imread(str(out_path))
KeyboardInterrupt


In [7]:
from multiprocessing import Pool
from tqdm import *

def worker(source):
    try:
        org_path = str(source).split('/')
        out_path = pathlib.Path(out, org_path[-3], org_path[-2], org_path[-1])

        if out_path.exists():
            img = cv2.imread(str(out_path))

            if img.shape == (1080, 1920, 3):
                return

        img = cv2.imread(str(source))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        if img.shape != (1080, 1920, 3):
            img = fix_image_size(img, source)

        out_path.parent.mkdir(parents=True, exist_ok=True)
        cv2.imwrite(str(out_path), img)
    except Exception as e:
        print('Error', source, e)

In [8]:
p = Pool(7)

pbar = tqdm(total=len(all_images_paths))
for i, _ in enumerate(p.imap_unordered(worker, all_images_paths)):
    pbar.update()

  0%|          | 1072/446054 [00:06<41:16, 179.70it/s] 

Error /data1/Raw/WallpaperPlay/initial-d-wallpapers/84744.jpg OpenCV(4.3.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'



  5%|▍         | 21589/446054 [02:34<37:28, 188.77it/s]  

Error /data1/Raw/WallpaperPlay/psychedelic-desktop-wallpapers/51504.jpg OpenCV(4.3.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'



 14%|█▍        | 61667/446054 [07:26<48:15, 132.74it/s]  

...

 87%|████████▋ | 387558/446054 [63:25:28<23:11:14,  1.43s/it]/home/andrew/anaconda3/envs/gogh-get-money/lib/python3.7/site-packages/PIL/Image.py:2835: DecompressionBombWarning: Image size (100000000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,
 88%|████████▊ | 392808/446054 [64:42:08<11:14:59,  1.31it/s] /home/andrew/anaconda3/envs/gogh-get-money/lib/python3.7/site-packages/PIL/Image.py:2835: DecompressionBombWarning: Image size (107952000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,
 91%|█████████ | 404852/446054 [67:40:53<6:01:23,  1.90it/s]  /home/andrew/anaconda3/envs/gogh-get-money/lib/python3.7/site-packages/PIL/Image.py:2835: DecompressionBombWarning: Image size (126786888 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,
 92%|█████████▏| 411264/446054 [69:14:38<13:14:27,  1.37s/it] /home/and

Error /data1/Raw/WallpaperAccess/solid-color-iphone/1248737.jpg Empty pixels when quantize.


 96%|█████████▌| 428841/446054 [73:29:45<52:28,  5.47it/s]   /home/andrew/anaconda3/envs/gogh-get-money/lib/python3.7/site-packages/PIL/Image.py:2835: DecompressionBombWarning: Image size (162000000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,
 98%|█████████▊| 436288/446054 [75:20:17<1:04:08,  2.54it/s] 

Error /data1/Raw/WallpaperAccess/plain-white/2077889.jpg Empty pixels when quantize.


 98%|█████████▊| 436688/446054 [75:26:47<2:53:56,  1.11s/it]/home/andrew/anaconda3/envs/gogh-get-money/lib/python3.7/site-packages/PIL/Image.py:2835: DecompressionBombWarning: Image size (144400000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,
100%|██████████| 446054/446054 [77:40:41<00:00,  1.88it/s]   

In [ ]:
p.close()
p.terminate()